In [1]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We start by loading the model from HuggingFace using the JATIC-toolbox.

In [2]:
from jatic_toolbox import load_model

model = load_model(
    provider="huggingface",
    model_name="Kaludi/food-category-classification-v2.0",
    task="image-classification"
)


2023-07-26 15:57:06.505928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 15:57:07.836539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/kyle-treubig/Code/jatic/armory/examples/.venv/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


The model returns a `HuggingFaceProbs` object type, but ART expects the model output to be the `y` tensor. So we have to adapt the model to produce the correct output.

In [3]:
from charmory.utils import adapt_jatic_image_classification_model_for_art

adapt_jatic_image_classification_model_for_art(model)

We then move the model to the target runtime defice and wrap it in an ART classifier to make it compatible with Armory/ART.

In [4]:
from art.estimators.classification import PyTorchClassifier

model.to(DEVICE)

classifier = PyTorchClassifier(
    model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(224, 224, 3),
    channels_first=False,
    nb_classes=12,
    clip_values=(0.0, 1.0),
)

Next we load the dataset from from HuggingFace using the JATIC-toolbox.

In [5]:
from jatic_toolbox import load_dataset

dataset = load_dataset(
    provider="huggingface",
    dataset_name="Kaludi/food-category-classification-v2.0",
    task="image-classification",
    split="validation",
)

Found cached dataset imagefolder (/home/kyle-treubig/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Loading cached processed dataset at /home/kyle-treubig/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-b82aa949443f821a.arrow


Since this dataset contains bad images that will result in errors during evaluation, we will apply a filter to the underlying HuggingFace dataset.

In [6]:
from transformers.image_utils import infer_channel_dimension_format
import numpy as np

def filter(sample):
    try:
        infer_channel_dimension_format(np.asarray(sample["image"]))
        return True
    except Exception as err:
        print(err)
        return False

print(f"Dataset length prior to filtering: {len(dataset)}")
dataset._dataset = dataset._dataset.filter(filter)
print(f"Dataset length after filtering: {len(dataset)}")

Loading cached processed dataset at /home/kyle-treubig/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-20e62b43d7f7d11a.arrow


Dataset length prior to filtering: 300
Dataset length after filtering: 280


Then prepare a transform for the data using the preprocessor that comes with the model.

In [7]:
from charmory.utils import create_jatic_image_classification_dataset_transform

transform = create_jatic_image_classification_dataset_transform(model.preprocessor)
dataset.set_transform(transform)

Then we create an Armory data generator around the dataset.

In [8]:
from charmory.data import JaticVisionDatasetGenerator

generator = JaticVisionDatasetGenerator(
    dataset=dataset,
    batch_size=16,
    epochs=1,
)

2023-07-26 15:57:29  0s WARNING  logging:callHandlers:1706 `tfds.core.add_checksums_dir` is deprecated. Refactor dataset in self-contained folders (`my_dataset/` folder containing my_dataset.py, my_dataset_test.py, dummy_data/, checksums.tsv). The checksum file will be automatically detected. More info at: https://www.tensorflow.org/datasets/add_dataset


Lastly we will define the Armory evaluation, including the attack and scenario to be run.

In [9]:
import art.attacks.evasion
from charmory.evaluation import (
    Attack,
    Dataset,
    Evaluation,
    Metric,
    Model,
    Scenario,
    SysConfig,
)
import charmory.scenarios.image_classification

eval_dataset = Dataset(
    name="food-category-classification",
    test_dataset=generator,
)

eval_model = Model(
    name="food-category-classification",
    model=classifier,
)

eval_attack = Attack(
    function=art.attacks.evasion.ProjectedGradientDescent,
    kwargs={
        "batch_size": 1,
        "eps": 0.031,
        "eps_step": 0.007,
        "max_iter": 20,
        "num_random_init": 1,
        "random_eps": False,
        "targeted": False,
        "verbose": False,
    },
    knowledge="white",
    use_label=True,
    type=None,
)

eval_scenario = Scenario(
    function=charmory.scenarios.image_classification.ImageClassificationTask,
    kwargs={},
)

eval_metric = Metric(
    profiler_type="basic",
    supported_metrics=["accuracy"],
    perturbation=["linf"],
    task=["categorical_accuracy"],
    means=True,
    record_metric_per_sample=False,
)

eval_sysconfig = SysConfig(
    gpus=["all"],
    use_gpu=True,
)

evaluation = Evaluation(
    name="food-category-classification",
    description="Food category classification from HuggingFace",
    author="Kaludi",
    dataset=eval_dataset,
    model=eval_model,
    attack=eval_attack,
    scenario=eval_scenario,
    metric=eval_metric,
    sysconfig=eval_sysconfig,
)

We now create an engine for the evaluation and run it.

In [10]:
from charmory.engine import Engine

engine = Engine(evaluation)
results = engine.run()
results

Evaluation: 100%|██████████| 18/18 [14:08<00:00, 47.15s/it]

2023-07-26 16:11:38 14m9s METRIC   armory.instrument.instrument:_write:743 benign_mean_categorical_accuracy on benign examples w.r.t. ground truth labels: 0.961
2023-07-26 16:11:38 14m9s METRIC   armory.instrument.instrument:_write:743 adversarial_mean_categorical_accuracy on adversarial examples w.r.t. ground truth labels: 0.489


{'armory_version': '23.4.0.post113+g0e7be67a.d20230713',
 'evaluation': Evaluation(name='food-category-classification', description='Food category classification from HuggingFace', model=Model(name='food-category-classification', model=art.estimators.classification.pytorch.PyTorchClassifier(model=ModelWrapper(
   (_model): HuggingFaceImageClassifier(
     (model): SwinForImageClassification(
       (swin): SwinModel(
         (embeddings): SwinEmbeddings(
           (patch_embeddings): SwinPatchEmbeddings(
             (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
           )
           (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
           (dropout): Dropout(p=0.0, inplace=False)
         )
         (encoder): SwinEncoder(
           (layers): ModuleList(
             (0): SwinStage(
               (blocks): ModuleList(
                 (0-1): 2 x SwinLayer(
                   (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True